In [36]:
import pandas as pd
import numpy as np
from sklearn.ensemble import IsolationForest
from sklearn.preprocessing import StandardScaler
import joblib


In [37]:
# Load main dataset (full population)
df_train = pd.read_csv("../data/synthetic/features_only.csv")

feature_cols = [
    "avgMonthlyIncome",
    "incomeCV",
    "expenseRatio",
    "emiRatio",
    "avgMonthlyBalance",
    "bounceCount"
]

X_train = df_train[feature_cols]

# Fit scaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)

# Train Isolation Forest
iso = IsolationForest(
    n_estimators=200,
    contamination=0.03,
    random_state=42,
    n_jobs=-1
)

iso.fit(X_train_scaled)

# Compute threshold ON TRAIN SET ONLY
train_scores = iso.decision_function(X_train_scaled)
threshold = np.percentile(train_scores, 3)

# Save models
joblib.dump(scaler, "../models/if_scaler.joblib")
joblib.dump(iso, "../models/isolation_forest.joblib")

print("Isolation Forest trained and saved")


Isolation Forest trained and saved


In [38]:
def apply_anomaly_detection(
    input_csv,
    output_csv,
    model_path="../models/isolation_forest.joblib",
    scaler_path="../models/if_scaler.joblib",
    threshold=threshold
):
    df = pd.read_csv(input_csv)
    
    feature_cols = [
        "avgMonthlyIncome",
        "incomeCV",
        "expenseRatio",
        "emiRatio",
        "avgMonthlyBalance",
        "bounceCount"
    ]
    
    X = df[feature_cols]
    
    scaler = joblib.load(scaler_path)
    iso = joblib.load(model_path)
    
    X_scaled = scaler.transform(X)
    
    df["anomaly_score"] = iso.decision_function(X_scaled)
    df["anomalyFlag"] = (df["anomaly_score"] < threshold).astype(int)
    
    df.to_csv(output_csv, index=False)
    
    print(f" Anomaly enriched: {output_csv}")
    
    return df 

In [39]:
df = apply_anomaly_detection(
    "../data/synthetic/features_only.csv",
    "../data/synthetic/features_with_anomaly.csv"
)

# Display results
anomaly_counts = df["anomalyFlag"].value_counts()
anomaly_pct = df["anomalyFlag"].value_counts(normalize=True) * 100

print("🔍 ANOMALY DETECTION RESULTS")
print(f"✅ Normal Transactions:  {anomaly_counts[0]:,} ({anomaly_pct[0]:.2f}%)")
print(f"🚨 Anomalies Detected:   {anomaly_counts[1]:,} ({anomaly_pct[1]:.2f}%)")
print(f"📊 Total Records:        {len(df):,}")

 Anomaly enriched: ../data/synthetic/features_with_anomaly.csv
🔍 ANOMALY DETECTION RESULTS
✅ Normal Transactions:  29,100 (97.00%)
🚨 Anomalies Detected:   900 (3.00%)
📊 Total Records:        30,000


In [40]:
apply_anomaly_detection(
    "../data/synthetic/features_only.csv",
    "../data/synthetic/features_with_anomaly.csv"
)

 Anomaly enriched: ../data/synthetic/features_with_anomaly.csv


,avgMonthlyIncome,incomeCV,expenseRatio,emiRatio,avgMonthlyBalance,bounceCount,accountAgeMonths,anomaly_score,anomalyFlag
0,315210.90,0.050,0.314,0.121,67185.96,0,100,0.153441,0
1,483386.74,0.038,0.292,0.125,210459.98,0,113,0.119100,0
2,488210.14,0.039,0.386,0.197,520559.06,1,105,0.039216,0
3,350279.88,0.040,0.408,0.129,365002.37,0,108,0.132264,0
4,449978.33,0.045,0.343,0.189,201947.71,0,90,0.146475,0
...,...,...,...,...,...,...,...,...,...
29995,485937.38,0.571,0.811,0.467,427895.51,1,54,0.031222,0
29996,570675.35,0.417,0.871,0.452,135421.36,3,51,0.023420,0
29997,411513.96,0.424,0.852,0.360,451633.99,1,57,0.062743,0
29998,440102.68,0.494,0.825,0.481,456177.44,1,25,0.040935,0
